<a href="https://colab.research.google.com/github/poojaneuusa/NLP-Projects/blob/main/Quora_Questions_Suggester.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
!pip install -U sentence-transformers
import torch
from sentence_transformers import SentenceTransformer, util
import os
import time
import pandas as pd
import requests

torch.__version__

url = 'http://qim.fs.quoracdn.net/quora_duplicate_questions.tsv'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36'}

response = requests.get(url, headers=headers)

if response.status_code == 200:
    with open('quora_duplicate_questions.tsv', 'wb') as file:
        file.write(response.content)
    df = pd.read_csv('quora_duplicate_questions.tsv', sep='\t', nrows=20000)
    print(df.shape)
else:
    print(f"Failed to fetch data: {response.status_code}")

df.head()

corpus_senteces = list(set(df['question1'].tolist() + df['question2'].tolist()))
len(corpus_senteces)

model = SentenceTransformer('quora-distilbert-multilingual')

corpus_embeddings = model.encode(corpus_senteces, show_progress_bar=True, convert_to_tensor=True)

model._target_device

corpus_embeddings = corpus_embeddings.to(model._target_device)

while True:
  query = input("Please enter a question: ")

  if query == 'n':
    break

  question_embedding = model.encode(query, convert_to_tensor=True)
  hits = util.semantic_search(question_embedding, corpus_embeddings, top_k=5)
  hits = hits[0]
  print(hits)

  for hit in hits:
    print(hit['score'], ": ", corpus_senteces[hit['corpus_id']])

  print('\n\n\n\n')


Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113
(20000, 6)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.77k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1181 [00:00<?, ?it/s]

Please enter a question: what is python
[{'corpus_id': 31611, 'score': 0.926170289516449}, {'corpus_id': 33249, 'score': 0.9184613823890686}, {'corpus_id': 1958, 'score': 0.9140186309814453}, {'corpus_id': 17150, 'score': 0.9131041765213013}, {'corpus_id': 1124, 'score': 0.9106529951095581}]
0.926170289516449 :  What are some cool python scripts?
0.9184613823890686 :  What is the difference between () and [] in Python?
0.9140186309814453 :  What are the differences between Python 1, 2, 3?
0.9131041765213013 :  The world's largest dinosaur discovered is a titanosaur. How big was his poop?
0.9106529951095581 :  What is the best beginner friendly book on python?





Please enter a question: How can I start with ML?
[{'corpus_id': 27939, 'score': 0.9037398099899292}, {'corpus_id': 28547, 'score': 0.9019578695297241}, {'corpus_id': 1548, 'score': 0.8988298773765564}, {'corpus_id': 14644, 'score': 0.8949798941612244}, {'corpus_id': 24961, 'score': 0.8946409821510315}]
0.9037398099899292 :  

KeyboardInterrupt: Interrupted by user